# Métodos de clase con herencia

In [1]:
import iso6346

In [2]:
class ContenedorDeMercancias:
    
    # Definición de atributos de clase
    
    siguiente_codigo_de_serie = 1337
    
    @staticmethod
    def _crear_codigo_BIC(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6))
    
    @classmethod
    def _generar_codigo_de_serie(cls):
        
        resultado = cls.siguiente_codigo_de_serie 
        cls.siguiente_codigo_de_serie += 1
        return resultado
    
    # Crea un contenedor vacío
    @classmethod
    def crear_contenedor_vacio(cls, codigo_propietario):
        return cls(codigo_propietario, contenidos = [])
    
    # Crea un contenedor a partir de un codigo de propietario y unos items
    @classmethod
    def crear_contenedor_con_elementos(cls, codigo_propietario, contenidos):
        return cls(codigo_propietario, list(contenidos))
        
    
    def __init__(self, codigo_propietario, contenidos):
        
        # Definición de atributos de instancia
        
        self.codigo_propietario = codigo_propietario
        self.contenidos = contenidos
        self.bic = ContenedorDeMercancias._crear_codigo_BIC(
            codigo_propietario=codigo_propietario,
            codigo_de_serie=ContenedorDeMercancias._generar_codigo_de_serie()
        )

Los métodos de clase definidos en la clase base van a ser heredados por las subclases.

Además, los argumentos cls de dichos métodos van a ser establecidos de forma apropiada.

Veamos un ejemplo:

In [3]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")

In [4]:
r1 = ContenedorDeMercanciasRefrigerado.crear_contenedor_vacio("YML")

In [5]:
r1

Como podemos ver, el contenedor creado es del tipo de la subclase, ya que ha heredado el método de clase y cls ha sido sustituido por el nombre de la subclase a la hora de llamar al método.

El otro método factory funciona igual:

In [6]:
r2 = ContenedorDeMercanciasRefrigerado.crear_contenedor_con_elementos("YML", ["fish"])

In [7]:
r2

Ahora vamos a añadir una temperatura para cada contenedor refrigerado como un atributo de instancia.

In [8]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS: 4.0
        
    def __init__(self, codigo_propietario, contenidos, celsius):
        super().__init__(codigo_propietario, contenidos)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        self.celsius = celsius
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")

In [9]:
try:
    r3 = ContenedorDeMercanciasRefrigerado.crear_contenedor_con_elementos("YML", ["fish"])
except Exception as error:
    print(error)

__init__() missing 1 required positional argument: 'celsius'


El problema está en que ahora los métodos factory no sirven, ya que el inicializador de la subclase ha cambiado y requiere de un argumento adicional: **celsius**.

Sin embargo, esto puede ser resuelto usando **kwargs**. 

Vamos a añadirlo al inicializador de la clase base y por lo tanto modificar su uso en los dos métodos de clase factory:

In [10]:
class ContenedorDeMercancias:
    
    # Definición de atributos de clase
    
    siguiente_codigo_de_serie = 1337
    
    @staticmethod
    def _crear_codigo_BIC(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6))
    
    @classmethod
    def _generar_codigo_de_serie(cls):
        
        resultado = cls.siguiente_codigo_de_serie 
        cls.siguiente_codigo_de_serie += 1
        return resultado
    
    # Crea un contenedor vacío
    @classmethod
    def crear_contenedor_vacio(cls, codigo_propietario, **kwargs):
        return cls(codigo_propietario, contenidos = [], **kwargs)
    
    # Crea un contenedor a partir de un codigo de propietario y unos items
    @classmethod
    def crear_contenedor_con_elementos(cls, codigo_propietario, contenidos, **kwargs):
        return cls(codigo_propietario, list(contenidos), **kwargs)
        
    
    def __init__(self, codigo_propietario, contenidos, **kwargs):
        
        # Definición de atributos de instancia
        
        self.codigo_propietario = codigo_propietario
        self.contenidos = contenidos
        self.bic = ContenedorDeMercancias._crear_codigo_BIC(
            codigo_propietario=codigo_propietario,
            codigo_de_serie=ContenedorDeMercancias._generar_codigo_de_serie()
        )

También vamos a modificar el inicializador de la subclase para obligar a **celsius** a ser un argumento de clave obligatoriamente:

In [11]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
        
    def __init__(self, codigo_propietario, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        self.celsius = celsius
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")

Ahora, el argumento celsius va a ser aceptado por el constructor de la subclase y enviado al constructor de la clase base, que se encuentra dentro del anterior

In [12]:
try:
    r3 = ContenedorDeMercanciasRefrigerado.crear_contenedor_con_elementos("YML", ["fish"], celsius=2.0)
except Exception as error:
    print(error)

In [13]:
r3

In [14]:
r3.celsius

2.0

In [15]:
r3.bic

'YMLU0013374'

Sin embargo, el diseño que hemos hecho de la clase tiene un problema, y es que nos permite saltarnos la restricción asociada a la temperatura:

In [16]:
r3.celsius = 11

In [17]:
r3.celsius

11

Esto lo solucionaremos en el próximo notebook, donde veremos las ***properties***.